In [1]:
using Random
using Distributions
using DataFrames, CSV
using FLoops

# compile local files
include("statistics.jl")
include("simulation.jl")
include("data.jl")
include("util.jl")

using .TestStatistics
using .Simulation
using .Data

In [19]:
"""
CONFIG
"""
alpha = 0.05

# data
B  = 10    # num. coverage probabilities per boxplot
S  = 1000   # num. samples per coverage probability
nx = 6      # size of group 1
ny = 6      # size of group 2
px, py = partition(nx, ny)
dtype = Float32

# distribution settings
Random.seed!(123)

distrTypeX = Normal
muX = dtype.(rand(Uniform(0, 1), B))
sdX = 2 * muX
paramsX = (muX, sdX)

distrTypeY = Normal
paramsY = paramsX

(Float32[0.90629965, 0.44349372, 0.74567336, 0.51208305, 0.2538491, 0.33415154, 0.42732787, 0.8675472, 0.58191234, 0.3114475], Float32[1.8125993, 0.88698745, 1.4913467, 1.0241661, 0.5076982, 0.6683031, 0.85465574, 1.7350944, 1.1638247, 0.622895])

In [ ]:
"""
distrTypeX = Gamma
shapeX = dtype.(rand(Uniform(1, 5), B))
scaleX = 4 * shapeX
paramsX = (shapeX, scaleX)

distrTypeY = Gamma
shapeY = dtype.(rand(Uniform(1, 5), B))
scaleY = 4 * shapeY
paramsY = (shapeY, scaleY)



distrTypeX = Exponential
lambdaX = dtype.(rand(Uniform(1, 20), B))
paramsX = (lambdaX,)

distrTypeY = Exponential
lambdaY = dtype.(rand(Uniform(1, 20), B))
paramsY = (lambdaY,)



distrTypeX = LogNormal
muX = dtype.(rand(Uniform(0, 1), B))
sdX = 2 * muX
paramsX = (muX, sdX)

distrTypeY = LogNormal
muY = dtype.(rand(Uniform(0, 1), B))
sdY = 2 * muY
paramsY = (muY, sdY)
"""

In [20]:
# generate data
Random.seed!(123)
x, y, wide, narrow, deltas, distrX, distrY = generateData(B, S, nx, ny, true,
                                                          distrTypeX, paramsX,
                                                          distrTypeY, paramsY)
@show size(x)
@show size(y)
@show size(wide)
@show size(narrow)
@show deltas[1]
@show distrX[1]
@show distrY[1]
x[1:5]

size(x) = (10, 1000, 6)
size(y) = (10, 1000, 6)
size(wide) = (10, 1000)
size(narrow) = (10, 1000)
deltas[1] = 0.0f0
distrX[1] = Normal{Float32}(μ=0.90629965f0, σ=1.8125993f0)
distrY[1] = Normal{Float32}(μ=0.90629965f0, σ=1.8125993f0)


5-element Vector{Float32}:
 -0.26415128
  0.28020507
  1.7277128
  1.6068726
 -0.5532921

In [21]:
# wrap constant arguments into an iterable object
# this allows each batch to have local access

struct Args
    px::AbstractMatrix{Int}
    py::AbstractMatrix{Int}
    pooled::Bool
    alpha::Float32
    alt_lo::Alternative
    alt_hi::Alternative
end

Base.Broadcast.broadcastable(obj::Args) = Ref(obj)  # make objects of this type iterable

In [22]:
basesize = ceil(Int, B / Threads.nthreads())

@time for isTwoSided in [true, false]
    alpha_temp = alpha
    alt_lo = alt_hi = twoSided
    
    if !isTwoSided
        alpha_temp = alpha / 2
        alt_lo = greater
        alt_hi = smaller
    end
        
    for pooled in [true, false]
        args = Args(px, py, pooled, alpha_temp, alt_lo, alt_hi)
        
        results = Vector{Any}(undef, B)
        @floop ThreadedEx(basesize = basesize) for b in 1:B
            results[b] = coverage(x[b,:,:], y[b,:,:], wide[b,:], narrow[b,:], deltas[b], args)
        end
        
        # convert results to DataFrame
        probs  = [x for (x, _) in results]
        widths = [x for (_, x) in results]
        df = DataFrame(prob=probs, width=widths, distrX=distrX, distrY=distrY)

        # save DataFrame as .csv
        filename = (isTwoSided ? "twoSided" : "oneSided") * "_" *
                   (pooled ? "" : "un") * "pooled" * "_" * string(args.alpha) * ".csv"
        
        #CSV.write("../results/" * filename, df)
        CSV.write(filename, df)
    end
end

 13.158914 seconds (43.16 M allocations: 77.853 GiB, 58.77% gc time, 0.86% compilation time)


In [ ]:
args = Args(px, py, true, 0.05, twoSided, twoSided)
results = Vector{Any}(undef, B)
for b in 1:B
    results[b] = coverage(x[b,:,:], y[b,:,:], wide[b,:], narrow[b,:], deltas[b], args)
end
results